In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Danh sách URL của các trang danh mục
category_pages = [f"https://bdshanoi.com.vn/can-ho/next-page-{i}.html" for i in range(1, 16)]
article_links = []

for page_url in category_pages:
    response = requests.get(page_url)

    soup = BeautifulSoup(response.text, "html.parser")

    for a_tag in soup.select("a[href^='/ban-can-ho-'], a[href^='/can-ho-']"):  
        href = a_tag["href"]
        if href.count("/") > 1:
            link = "https://bdshanoi.com.vn" + a_tag["href"]
            if link not in article_links:
                article_links.append(link)
            if len(article_links) >= 200:
                break
    if len(article_links) >= 200:
        break

# Hàm xử lý thông tin thành dictionary
def parse_infos(info_list):
    info_dict = {}
    for item in info_list:
        if ":" in item:
            key, value = item.split(":", 1)
            info_dict[key.strip()] = value.strip()
    return info_dict

all_attributes = set()
data = []

# Duyệt từng link bài chi tiết và trích xuất thông tin
for url in article_links:
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")

    div_title = soup.find("div", class_="prohead")
    title = div_title.find("h1").text.strip() 

    div_id = soup.find("div", class_="proleft")
    id = div_id.find("span").text.strip() 

    div_price = soup.find("div", class_="proright")
    price = div_price.find("span").text.strip() if div_price and div_price.find("span") else "Vui lòng liên hệ"

    div_info = soup.find("ul", class_="ultech")
    infos = [info.text.strip() for info in div_info.findAll("li")] if div_info else []
    parsed_info = parse_infos(infos)
    all_attributes.update(parsed_info.keys())

    row = {"title": title, "id": id, "price": price}
    row.update(parsed_info)
    data.append(row)

# Chuẩn hóa dữ liệu với tất cả thuộc tính
for row in data:
    for attr in all_attributes:
        row.setdefault(attr, None)

# Chuyển dữ liệu thành DataFrame và lưu vào CSV
df = pd.DataFrame(data)
df.to_csv("real_estate_data.csv", index=False)


# Hàm xử lý thông tin thành dictionary
def parse_infos(info_list):
    info_dict = {}
    for item in info_list:
        if ":" in item:
            key, value = item.split(":", 1)
            info_dict[key.strip()] = value.strip()
    return info_dict

all_attributes = set()
data = []

# Duyệt từng link bài chi tiết và trích xuất thông tin
for url in article_links:
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")

    div_title = soup.find("div", class_="prohead")
    title = div_title.find("h1").text.strip() 

    div_id = soup.find("div", class_="proleft")
    id = div_id.find("span").text.strip() 

    div_price = soup.find("div", class_="proright")
    price = div_price.find("span").text.strip() if div_price and div_price.find("span") else "Vui lòng liên hệ"

    div_info = soup.find("ul", class_="ultech")
    infos = [info.text.strip() for info in div_info.findAll("li")] if div_info else []
    parsed_info = parse_infos(infos)
    all_attributes.update(parsed_info.keys())

    row = {"title": title, "id": id, "price": price}
    row.update(parsed_info)
    data.append(row)

# Chuẩn hóa dữ liệu với tất cả thuộc tính
for row in data:
    for attr in all_attributes:
        row.setdefault(attr, None)

# Chuyển dữ liệu thành DataFrame và lưu vào CSV
df = pd.DataFrame(data)
df.to_csv("real_estate_data.csv", index=False)
